# Pipeline Principal — Inferencia por Cluster

Flujo completo:
1. Configurar parámetros (cluster, número de cajeros, horizonte)
2. Cargar datos históricos
3. Clasificar cajeros y seleccionar los del cluster elegido
4. Cargar el modelo entrenado
5. Ejecutar inferencia
6. Ver resultados por cajero y resumen final

**Clusters disponibles:** `normal_estable`, `normal_con_picos`, `event_driven`, `grande_y_estable`

In [ ]:
import sys
from pathlib import Path

PROJECT_ROOT = Path().resolve().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

DATA_PATH   = PROJECT_ROOT / "insumos" / "df_general.parquet"
MODELO_PATH = PROJECT_ROOT / "models"  / "neuralprophet.np"

print("PROJECT_ROOT:", PROJECT_ROOT)
print("Datos:       ", DATA_PATH,   "— existe:", DATA_PATH.exists())
print("Modelo:      ", MODELO_PATH, "— existe:", MODELO_PATH.exists())

## Configuración — modificar aquí

In [ ]:
# ============================================================
# PARÁMETROS DEL USUARIO
# ============================================================

# Cluster a analizar.
# Opciones: "normal_estable" | "normal_con_picos" | "event_driven" | "grande_y_estable"
CLUSTER = "normal_estable"

# Número de cajeros a seleccionar de ese cluster
N_CAJEROS = 3

# Modo de selección de cajeros
# Opciones: "aleatorio" | "mayor_mediana" | "mayor_ratio"
MODO_SELECCION = "aleatorio"

# Días a predecir hacia el futuro
HORIZONTE = 30

print(f"Cluster:       {CLUSTER}")
print(f"N cajeros:     {N_CAJEROS}")
print(f"Modo:          {MODO_SELECCION}")
print(f"Horizonte:     {HORIZONTE} días")

## Imports

In [ ]:
import pandas as pd
import numpy as np

from src.segmentation import (
    construir_resumen_estructural,
    aplicar_reglas_precluster,
    seleccionar_cajeros_por_cluster
)
from src.inference import inferencia_por_lista

print("Imports OK")

## 1. Cargar datos históricos

In [ ]:
df = pd.read_parquet(DATA_PATH)

print(f"Filas: {len(df):,}")
print(f"Columnas: {list(df.columns)}")
print(f"Cajeros únicos: {df['cajero'].nunique():,}")
print(f"Período: {df['fecha'].min()} → {df['fecha'].max()}")

## 2. Clasificar cajeros y seleccionar cluster

In [ ]:
resumen = construir_resumen_estructural(df)
df_clasificados = aplicar_reglas_precluster(resumen)

print("Distribución de clusters:")
print(df_clasificados["etiqueta"].value_counts().to_string())
print()

df_cluster_sel = seleccionar_cajeros_por_cluster(
    df_clasificados,
    cluster=CLUSTER,
    n=N_CAJEROS,
    modo=MODO_SELECCION
)

lista_cajeros = df_cluster_sel["cajero"].tolist()

print(f"Cajeros seleccionados del cluster '{CLUSTER}':")
print(df_cluster_sel[["cajero", "dias_obs", "mediana", "ratio_p95_mediana", "etiqueta"]].to_string(index=False))

## 3. Cargar modelo entrenado

In [ ]:
if not MODELO_PATH.exists():
    raise FileNotFoundError(
        f"No se encontró el modelo en:\n  {MODELO_PATH}\n"
        "Ejecuta primero el notebook de entrenamiento: Scripts/prueba_neuro_prophet.ipynb"
    )

print(f"Modelo encontrado: {MODELO_PATH}")
print("El modelo se cargará durante la inferencia.")

## 4. Ejecutar inferencia

In [ ]:
print(f"Iniciando inferencia para {len(lista_cajeros)} cajero(s) del cluster '{CLUSTER}'...")
print(f"Horizonte: {HORIZONTE} días\n")

resultados = inferencia_por_lista(
    lista_cajeros=lista_cajeros,
    df_historico=df,
    ruta_modelo=MODELO_PATH,
    horizonte=HORIZONTE
)

print(f"\nInferencia completada: {len(resultados)} cajero(s) con predicciones.")

## 5. Resultados por cajero

In [ ]:
if not resultados:
    print("No hay resultados. Revisa que los cajeros tengan al menos 150 días de historial.")
else:
    for cajero_id, datos in resultados.items():
        pred = datos["predicciones_futuras"]
        media_hist = datos["media_historica"]
        n_dias = datos["n_dias_historicos"]

        yhat_col = "yhat1" if "yhat1" in pred.columns else pred.columns[-1]
        media_pred = round(pred[yhat_col].mean(), 2)
        max_pred   = round(pred[yhat_col].max(), 2)
        min_pred   = round(pred[yhat_col].min(), 2)

        cambio_pct = round(((media_pred - media_hist) / media_hist) * 100, 1) if media_hist != 0 else 0
        tendencia  = f"▲ +{cambio_pct}%" if cambio_pct >= 0 else f"▼ {cambio_pct}%"

        print("=" * 55)
        print(f"  Cajero: {cajero_id}  |  Cluster: {CLUSTER}")
        print("=" * 55)
        print(f"  Días históricos:    {n_dias}")
        print(f"  Media histórica:    {media_hist:>12,.0f}")
        print(f"  Media predicha:     {media_pred:>12,.0f}  {tendencia}")
        print(f"  Máx predicho:       {max_pred:>12,.0f}")
        print(f"  Mín predicho:       {min_pred:>12,.0f}")
        print()
        print("  Predicciones diarias:")
        print(pred[["ds", yhat_col]].to_string(index=False))
        print()

## 6. Resumen final

In [ ]:
if not resultados:
    print("Sin resultados para mostrar.")
else:
    filas = []
    for cajero_id, datos in resultados.items():
        pred = datos["predicciones_futuras"]
        media_hist = datos["media_historica"]
        yhat_col = "yhat1" if "yhat1" in pred.columns else pred.columns[-1]

        media_pred = pred[yhat_col].mean()
        cambio_pct = round(((media_pred - media_hist) / media_hist) * 100, 1) if media_hist != 0 else 0

        filas.append({
            "Cajero":            cajero_id,
            "Cluster":           CLUSTER,
            "Días históricos":   datos["n_dias_historicos"],
            "Media histórica":   round(media_hist, 0),
            "Media predicha":    round(media_pred, 0),
            "Máx predicho":      round(pred[yhat_col].max(), 0),
            "Mín predicho":      round(pred[yhat_col].min(), 0),
            "Cambio %":          f"{'+' if cambio_pct >= 0 else ''}{cambio_pct}%",
        })

    df_resumen = pd.DataFrame(filas)

    print(f"RESUMEN — Cluster '{CLUSTER}' — Horizonte {HORIZONTE} días")
    print("=" * 80)
    print(df_resumen.to_string(index=False))
    print()

    media_global = df_resumen["Media predicha"].mean()
    media_hist_global = df_resumen["Media histórica"].mean()
    cambio_global = round(((media_global - media_hist_global) / media_hist_global) * 100, 1) if media_hist_global != 0 else 0

    print(f"Cajeros procesados:      {len(resultados)}")
    print(f"Media histórica global:  {media_hist_global:,.0f}")
    print(f"Media predicha global:   {media_global:,.0f}  ({'+' if cambio_global >= 0 else ''}{cambio_global}%)")